# Final Dataset Creation

This is the first step in our work. This notebook uses the LIDC-IDRI dataset in order to create a new folder, in which only the files we are going to use are stored (slices and their respective mask). 

The way it works is very simple! It iterates through each scan and, for each anotated nodule, it creates a folder that contains sub-folders for each annotation of said nodule, in which it saves the slices and respective masks of the annotation. This is done so these files can be used as input for the PyRadiomics feature extractor, in the next step.

The new folder has the following tree structure:

FINAL_DATASET<br>
|<br>
| ---- SCAN_x<br>
|&emsp;&emsp;&emsp;&emsp;|<br>
|&emsp;&emsp;&emsp;&emsp;| ---- ANNOTATION_x<br>
|&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;&emsp;|<br>
|&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;&emsp;| ---- slice_1<br>
|&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;&emsp;| ---- mask_1<br>
|&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;&emsp;| ---- slice_2<br>
|&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;&emsp;| ---- mask_2<br>
|&emsp;&emsp;&emsp;&emsp;|&emsp;&emsp;&emsp;&emsp;| ---- ...<br>
...&emsp;&emsp;&emsp; ...&emsp;&emsp;&emsp; ...

# 

# 0. Libraries

In [1]:
import os
import numpy as np
import pylidc as pl
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import nibabel as nib

# 

# 1. Dataset Creation

###### First, we define the directory of the dataset we are about to create

In [2]:
#current directory
root_dir = os.getcwd()
print("Root directory: ", root_dir, "\n")


#create the new dataset directory if it doesn't exist
final_dataset_dir = os.path.join(root_dir, 'FINAL_DATASET')
if not os.path.exists(final_dataset_dir):
    os.makedirs(final_dataset_dir)
print("Final dataset directory: ", final_dataset_dir)

Root directory:  /home/miguel/Desktop/Universidade/Ano4/Semestre1/LaboratorioIACD/LIACD - Trabalho1/WIP 

Final dataset directory:  /home/miguel/Desktop/Universidade/Ano4/Semestre1/LaboratorioIACD/LIACD - Trabalho1/WIP/FINAL_DATASET


###### Then, we analyse the scans

In [3]:
#get all the scans
scans = pl.query(pl.Scan).all()

#list that will save all the unique patient ids
lista = []

#iterate through each scan, and save the patient id of the scan to the list we created
for scan in scans:
    lista.append(scan.patient_id)

#compare total of scans to unique scans
print("Total of scans: ", len(scans))
print("Total of unique scans: ", len(np.unique(lista)), "\n")

#iterate through each scan, and print it
for scan in scans:
    print(scan)

Total of scans:  1018
Total of unique scans:  1010 

Scan(id=1,patient_id=LIDC-IDRI-0078)
Scan(id=2,patient_id=LIDC-IDRI-0069)
Scan(id=3,patient_id=LIDC-IDRI-0079)
Scan(id=4,patient_id=LIDC-IDRI-0101)
Scan(id=5,patient_id=LIDC-IDRI-0110)
Scan(id=6,patient_id=LIDC-IDRI-0115)
Scan(id=7,patient_id=LIDC-IDRI-0132)
Scan(id=8,patient_id=LIDC-IDRI-0136)
Scan(id=9,patient_id=LIDC-IDRI-0150)
Scan(id=10,patient_id=LIDC-IDRI-0151)
Scan(id=11,patient_id=LIDC-IDRI-0154)
Scan(id=12,patient_id=LIDC-IDRI-0001)
Scan(id=13,patient_id=LIDC-IDRI-0002)
Scan(id=14,patient_id=LIDC-IDRI-0003)
Scan(id=15,patient_id=LIDC-IDRI-0004)
Scan(id=16,patient_id=LIDC-IDRI-0005)
Scan(id=17,patient_id=LIDC-IDRI-0006)
Scan(id=18,patient_id=LIDC-IDRI-0007)
Scan(id=19,patient_id=LIDC-IDRI-0008)
Scan(id=20,patient_id=LIDC-IDRI-0009)
Scan(id=21,patient_id=LIDC-IDRI-0010)
Scan(id=22,patient_id=LIDC-IDRI-0011)
Scan(id=23,patient_id=LIDC-IDRI-0012)
Scan(id=24,patient_id=LIDC-IDRI-0013)
Scan(id=25,patient_id=LIDC-IDRI-0014)
Scan(i

###### Finally, we iterate through each scan, retrieving the nodules (present in each scan) and their respective annotations

In [4]:
#iterate through each scan
for scan in scans:
    
    
    #patient id for current scan
    scan_id = "SCAN_" + str(scan.id)   
    
    
    #print current scan
    print("Scan", scan.id)
    
    
    #create a patient-specific folder in the final dataset directory
    scan_dir = os.path.join(final_dataset_dir, scan_id)
    if not os.path.exists(scan_dir):
        os.makedirs(scan_dir)
        
        
    #get the nodules (group of annotations) for current scan
    nodules = scan.cluster_annotations()
    
    
    #iterate through the nodules of current scan
    for i, nodule in enumerate(nodules):
        
        
        #if the nodule has 0 annotations, skip it!
        if len(nodule)  == 0:
            continue
                
        
        #iterate through the annotations of the current nodule
        for j, annotation in enumerate(nodule):
            
            
            #get annotation id
            annotation_id = "ANNOTATION_" + str(annotation.id)
            
        
            #create an annotation-specific folder in the scan directory
            annotation_dir = os.path.join(scan_dir, annotation_id)
            if not os.path.exists(annotation_dir):
                os.makedirs(annotation_dir)
                
            
            #convert the 3d scan to a numpy array
            vol = annotation.scan.to_volume()
            
            
            #print the current annotation
            print("   Annotation", annotation.id, "=", annotation)
            

            #get the mask; if mask.shape is (10, 50, 50) then the nodule's mask spans 10 slices in the z-direction, with each slice being 50x50 pixels in size in the y and x dimensions; the values inside the mask are True (or 1) where the nodule is present, and False (or 0) elsewhere
            mask = annotation.boolean_mask()
            print("      Mask shape = ", mask.shape)
            
            
            #get the bounding box
            #BBOX IS PRESENTED LIKE (slice(300, 327, None), slice(318, 346, None), slice(23, 29, None))
            #First Slice (slice(300, 327, None)):
            #    This represents the z-axis (depth or slices in the CT scan).
            #    It indicates that the nodule spans from slice 300 to slice 326 (327 is exclusive) along the z-axis. This means the nodule is present in slices 300, 301, ..., up to 326.
            #Second Slice (slice(318, 346, None)):
            #    This corresponds to the y-axis (rows within each slice).
            #    The nodule spans from row 318 to row 345 (346 is exclusive) within the slices.
            #Third Slice (slice(23, 29, None)):
            #    This is for the x-axis (columns within each slice).
            #    It indicates the nodule is present from column 23 to column 28 (29 is exclusive) in the slices.
            bbox = annotation.bbox()
            slices_span = bbox[2]
            
            
            #get the bounding box dimensions
            bbox_dims = annotation.bbox_dims()
            print("      Bbox dims = ", bbox_dims)
            
                        
            #get the number of slices values for current annotation
            slices = mask.shape[2]
            print("      Number of slices = ", slices)
            print("      slice(start, stop, step) = ", slices_span, "\n")
            
            
            #padding = [(30,10), (10,25), (0,0)]
            
            
            #iterate through the slices in the current annotation
            for slice_index in range(slices):
                
                
                #print current slice
                print("      Saving slice ", slice_index+1, "/", slices)
                
                
                #extract the volume and mask for the current slice
                slice_volume = vol[bbox][:, :, slice_index] 
                slice_mask = mask[:, :, slice_index]  
                
                
                #create NIfTI images for the slice and mask
                slice_nifti = nib.Nifti1Image(slice_volume, np.eye(4))  #identity affine for now (keep it that way!)
                mask_nifti = nib.Nifti1Image(slice_mask.astype(np.uint8), np.eye(4))  #ensure mask is in uint8 format
        
                
                #construct directories for saving
                slice_dir = os.path.join(annotation_dir, f"slice_{slice_index + 1}.nii.gz")
                mask_dir = os.path.join(annotation_dir, f"mask_{slice_index + 1}.nii.gz")
                
                
                # Save the slice and mask as .nii.gz files
                nib.save(slice_nifti, slice_dir)
                nib.save(mask_nifti, mask_dir)
           



                """
                #VISUALIZACAO FORMA 1
                fig, ax = plt.subplots(1, 2, figsize=(10, 5))  # Adjust figsize as needed
                # Display the full volume slice
                ax[0].imshow(vol[bbox][:, :, slice_index], cmap=plt.cm.gray, aspect='auto')
                ax[0].set_title('Slice')
                ax[0].axis('off')
                # Display the full mask slice
                ax[1].imshow(mask[:, :, slice_index], cmap=plt.cm.gray, aspect='auto')
                ax[1].set_title('Mask')
                ax[1].axis('off')
                plt.tight_layout()
                plt.show()
                """
                
            
                """
                #VISUALIZACAO FORMA 2
                fig,ax = plt.subplots(1,2,figsize=(5,3))
                ax[0].imshow(vol[:,:,2], cmap=plt.cm.gray)
                ax[0].axis('off')
                ax[1].imshow(mask[:,:,2], cmap=plt.cm.gray)
                ax[1].axis('off')
                plt.tight_layout()
                #plt.savefig("../images/mask_bbox.png", bbox_inches="tight")
                plt.show()
                """
                
            print()
                
                
            
        print()
        
        
    print("------\n")    
    


Scan 1
Loading dicom files ... This may take a moment.
   Annotation 2 = Annotation(id=2,scan_id=1)
      Mask shape =  (27, 28, 6)
      Bbox dims =  [16.9  17.55 15.  ]
      Number of slices =  6
      slice(start, stop, step) =  slice(23, 29, None) 

      Saving slice  1 / 6
      Saving slice  2 / 6
      Saving slice  3 / 6
      Saving slice  4 / 6
      Saving slice  5 / 6
      Saving slice  6 / 6

Loading dicom files ... This may take a moment.
   Annotation 6 = Annotation(id=6,scan_id=1)
      Mask shape =  (33, 40, 6)
      Bbox dims =  [20.8  25.35 15.  ]
      Number of slices =  6
      slice(start, stop, step) =  slice(23, 29, None) 

      Saving slice  1 / 6
      Saving slice  2 / 6
      Saving slice  3 / 6
      Saving slice  4 / 6
      Saving slice  5 / 6
      Saving slice  6 / 6

Loading dicom files ... This may take a moment.
   Annotation 10 = Annotation(id=10,scan_id=1)
      Mask shape =  (32, 42, 5)
      Bbox dims =  [20.15 26.65 12.  ]
      Number of s

   Annotation 21 = Annotation(id=21,scan_id=2)
      Mask shape =  (25, 35, 7)
      Bbox dims =  [17.784 25.194 12.   ]
      Number of slices =  7
      slice(start, stop, step) =  slice(80, 87, None) 

      Saving slice  1 / 7
      Saving slice  2 / 7
      Saving slice  3 / 7
      Saving slice  4 / 7
      Saving slice  5 / 7
      Saving slice  6 / 7
      Saving slice  7 / 7


------

Scan 3
Loading dicom files ... This may take a moment.
   Annotation 23 = Annotation(id=23,scan_id=3)
      Mask shape =  (25, 18, 5)
      Bbox dims =  [16.752 11.866 12.   ]
      Number of slices =  5
      slice(start, stop, step) =  slice(67, 72, None) 

      Saving slice  1 / 5
      Saving slice  2 / 5
      Saving slice  3 / 5
      Saving slice  4 / 5
      Saving slice  5 / 5

Loading dicom files ... This may take a moment.
   Annotation 24 = Annotation(id=24,scan_id=3)
      Mask shape =  (23, 19, 5)
      Bbox dims =  [15.356 12.564 12.   ]
      Number of slices =  5
      slice(sta

   Annotation 46 = Annotation(id=46,scan_id=7)
      Mask shape =  (21, 15, 1)
      Bbox dims =  [10.7421875   7.51953125  0.        ]
      Number of slices =  1
      slice(start, stop, step) =  slice(93, 94, None) 

      Saving slice  1 / 1


Loading dicom files ... This may take a moment.
   Annotation 39 = Annotation(id=39,scan_id=7)
      Mask shape =  (38, 50, 6)
      Bbox dims =  [19.87304688 26.31835938 15.        ]
      Number of slices =  6
      slice(start, stop, step) =  slice(95, 101, None) 

      Saving slice  1 / 6
      Saving slice  2 / 6
      Saving slice  3 / 6
      Saving slice  4 / 6
      Saving slice  5 / 6
      Saving slice  6 / 6

Loading dicom files ... This may take a moment.
   Annotation 40 = Annotation(id=40,scan_id=7)
      Mask shape =  (40, 62, 6)
      Bbox dims =  [20.94726562 32.76367188 15.        ]
      Number of slices =  6
      slice(start, stop, step) =  slice(95, 101, None) 

      Saving slice  1 / 6
      Saving slice  2 / 6
     

   Annotation 60 = Annotation(id=60,scan_id=8)
      Mask shape =  (12, 11, 2)
      Bbox dims =  [9.1953125 8.359375  3.       ]
      Number of slices =  2
      slice(start, stop, step) =  slice(125, 127, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2


Loading dicom files ... This may take a moment.
   Annotation 51 = Annotation(id=51,scan_id=8)
      Mask shape =  (8, 6, 2)
      Bbox dims =  [5.8515625 4.1796875 3.       ]
      Number of slices =  2
      slice(start, stop, step) =  slice(136, 138, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2

Loading dicom files ... This may take a moment.
   Annotation 59 = Annotation(id=59,scan_id=8)
      Mask shape =  (10, 8, 2)
      Bbox dims =  [7.5234375 5.8515625 3.       ]
      Number of slices =  2
      slice(start, stop, step) =  slice(136, 138, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2


------

Scan 9
Loading dicom files ... This may take a moment.
   Annotation 73 = Annotation(id=73

   Annotation 89 = Annotation(id=89,scan_id=13)
      Mask shape =  (41, 50, 23)
      Bbox dims =  [27.26564  33.400409 27.5     ]
      Number of slices =  23
      slice(start, stop, step) =  slice(176, 199, None) 

      Saving slice  1 / 23
      Saving slice  2 / 23
      Saving slice  3 / 23
      Saving slice  4 / 23
      Saving slice  5 / 23
      Saving slice  6 / 23
      Saving slice  7 / 23
      Saving slice  8 / 23
      Saving slice  9 / 23
      Saving slice  10 / 23
      Saving slice  11 / 23
      Saving slice  12 / 23
      Saving slice  13 / 23
      Saving slice  14 / 23
      Saving slice  15 / 23
      Saving slice  16 / 23
      Saving slice  17 / 23
      Saving slice  18 / 23
      Saving slice  19 / 23
      Saving slice  20 / 23
      Saving slice  21 / 23
      Saving slice  22 / 23
      Saving slice  23 / 23


------

Scan 14
Loading dicom files ... This may take a moment.
   Annotation 102 = Annotation(id=102,scan_id=14)
      Mask shape =  (36, 34, 8

...

   Annotation 6759 = Annotation(id=6759,scan_id=1006)
      Mask shape =  (7, 7, 2)
      Bbox dims =  [4.21875 4.21875 1.25   ]
      Number of slices =  2
      slice(start, stop, step) =  slice(136, 138, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2

Loading dicom files ... This may take a moment.
   Annotation 6775 = Annotation(id=6775,scan_id=1006)
      Mask shape =  (7, 7, 2)
      Bbox dims =  [4.21875 4.21875 1.25   ]
      Number of slices =  2
      slice(start, stop, step) =  slice(136, 138, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2

Loading dicom files ... This may take a moment.
   Annotation 6783 = Annotation(id=6783,scan_id=1006)
      Mask shape =  (9, 8, 2)
      Bbox dims =  [5.625    4.921875 1.25    ]
      Number of slices =  2
      slice(start, stop, step) =  slice(136, 138, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2


Loading dicom files ... This may take a moment.
   Annotation 6755 = Annotation(id=6755,scan_id=

   Annotation 6790 = Annotation(id=6790,scan_id=1007)
      Mask shape =  (17, 16, 18)
      Bbox dims =  [10.34375     9.69726562  8.5       ]
      Number of slices =  18
      slice(start, stop, step) =  slice(492, 510, None) 

      Saving slice  1 / 18
      Saving slice  2 / 18
      Saving slice  3 / 18
      Saving slice  4 / 18
      Saving slice  5 / 18
      Saving slice  6 / 18
      Saving slice  7 / 18
      Saving slice  8 / 18
      Saving slice  9 / 18
      Saving slice  10 / 18
      Saving slice  11 / 18
      Saving slice  12 / 18
      Saving slice  13 / 18
      Saving slice  14 / 18
      Saving slice  15 / 18
      Saving slice  16 / 18
      Saving slice  17 / 18
      Saving slice  18 / 18


------

Scan 1008
Loading dicom files ... This may take a moment.
   Annotation 6792 = Annotation(id=6792,scan_id=1008)
      Mask shape =  (10, 11, 4)
      Bbox dims =  [5.625 6.25  1.875]
      Number of slices =  4
      slice(start, stop, step) =  slice(207, 211, Non

   Annotation 6812 = Annotation(id=6812,scan_id=1013)
      Mask shape =  (14, 12, 2)
      Bbox dims =  [8.45 7.15 3.  ]
      Number of slices =  2
      slice(start, stop, step) =  slice(50, 52, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2

Loading dicom files ... This may take a moment.
   Annotation 6816 = Annotation(id=6816,scan_id=1013)
      Mask shape =  (11, 12, 2)
      Bbox dims =  [6.5  7.15 3.  ]
      Number of slices =  2
      slice(start, stop, step) =  slice(50, 52, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2

Loading dicom files ... This may take a moment.
   Annotation 6819 = Annotation(id=6819,scan_id=1013)
      Mask shape =  (11, 11, 2)
      Bbox dims =  [6.5 6.5 3. ]
      Number of slices =  2
      slice(start, stop, step) =  slice(50, 52, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2


Loading dicom files ... This may take a moment.
   Annotation 6813 = Annotation(id=6813,scan_id=1013)
      Mask shape =  (9, 10,

   Annotation 6834 = Annotation(id=6834,scan_id=1014)
      Mask shape =  (7, 7, 2)
      Bbox dims =  [3.867186 3.867186 2.      ]
      Number of slices =  2
      slice(start, stop, step) =  slice(70, 72, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2

Loading dicom files ... This may take a moment.
   Annotation 6841 = Annotation(id=6841,scan_id=1014)
      Mask shape =  (10, 8, 2)
      Bbox dims =  [5.800779 4.511717 2.      ]
      Number of slices =  2
      slice(start, stop, step) =  slice(70, 72, None) 

      Saving slice  1 / 2
      Saving slice  2 / 2


Loading dicom files ... This may take a moment.
   Annotation 6825 = Annotation(id=6825,scan_id=1014)
      Mask shape =  (15, 12, 4)
      Bbox dims =  [9.023434 7.089841 6.      ]
      Number of slices =  4
      slice(start, stop, step) =  slice(80, 84, None) 

      Saving slice  1 / 4
      Saving slice  2 / 4
      Saving slice  3 / 4
      Saving slice  4 / 4

Loading dicom files ... This may take a m

   Annotation 6859 = Annotation(id=6859,scan_id=1018)
      Mask shape =  (50, 47, 13)
      Bbox dims =  [38.18554688 35.84765625 36.        ]
      Number of slices =  13
      slice(start, stop, step) =  slice(68, 81, None) 

      Saving slice  1 / 13
      Saving slice  2 / 13
      Saving slice  3 / 13
      Saving slice  4 / 13
      Saving slice  5 / 13
      Saving slice  6 / 13
      Saving slice  7 / 13
      Saving slice  8 / 13
      Saving slice  9 / 13
      Saving slice  10 / 13
      Saving slice  11 / 13
      Saving slice  12 / 13
      Saving slice  13 / 13


------

